In [1]:
import sys

print(sys.executable)
print(sys.version)

/home/j-i13a103/.conda/envs/stt/bin/python
3.10.18 | packaged by conda-forge | (main, Jun  4 2025, 14:45:41) [GCC 13.3.0]


In [3]:
# GPU 디바이스 설정
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [3]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
Tesla V100-PCIE-32GB


# 마이크 테스트 - wav 파일 인식

In [2]:
# 필요한 라이브러리 로드

from faster_whisper import WhisperModel

/home/j-i13a103/.conda/envs/stt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import os

os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "true"
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [5]:
# 모델 로딩하기
model = WhisperModel("tiny", device="cpu", compute_type="int8")

In [6]:
# 음성 파일 넣어보기
data_path = "./청소년_상담_데이터/1.데이터/Training/01.원천데이터/0015.mp3"
# "한국어_아동_음성_데이터/01.데이터/1.Training/원천데이터_230421_add/TS_kor_formatted_04/2021-09-01/0011/K00030011-AFG33-L1N2D1-E-K0KK-00000678.wav"
# "./한국어_아동_음성_데이터/01.데이터/1.Training/원천데이터_230421_add/TS_kor_formatted_04/2021-09-01/0009/K00150009-AFG30-L1N2D4-E-K0KK-00000140.wav"

segments, info = model.transcribe(data_path, beam_size=5, word_timestamps=True, no_speech_threshold=0.4)

In [7]:
info

TranscriptionInfo(language='ko', language_probability=0.9937093257904053, duration=607.8171875, duration_after_vad=607.8171875, all_language_probs=[('ko', 0.9937093257904053), ('ja', 0.0027736094780266285), ('nn', 0.001543693128041923), ('en', 0.0006943519692867994), ('zh', 0.0005031463806517422), ('jw', 0.00012709395377896726), ('de', 7.253919466165826e-05), ('pl', 6.112403934821486e-05), ('th', 4.496052861213684e-05), ('haw', 4.275235187378712e-05), ('tr', 4.150625318288803e-05), ('km', 3.863889287458733e-05), ('hi', 3.8132293411763385e-05), ('si', 3.6083791201235726e-05), ('la', 3.240757723688148e-05), ('es', 2.9065429771435447e-05), ('ru', 2.4685889002284966e-05), ('fr', 2.45126175286714e-05), ('mn', 2.2260597688728012e-05), ('sn', 2.1465337340487167e-05), ('bn', 1.992914803849999e-05), ('pt', 1.3589695299742743e-05), ('ne', 9.71677673078375e-06), ('te', 9.471893463341985e-06), ('id', 6.646892870776355e-06), ('cy', 6.139432571217185e-06), ('my', 5.980928563076304e-06), ('bo', 5.574

In [8]:
dic_list = []
for segment in segments:
    no_speech_prob = segment.no_speech_prob
    words_with_timestamps = []
    for word in segment.words:
        _word = word.word
        _start = word.start
        _end = word.end
        dic_list.append([_word, _start, _end, no_speech_prob])

print(dic_list)

[[' 최근에', np.float64(0.0), np.float64(2.36), 0.05322595685720444], [' 아픈', np.float64(2.36), np.float64(2.68), 0.05322595685720444], [' 곳이', np.float64(2.68), np.float64(3.02), 0.05322595685720444], [' 있었네?', np.float64(3.02), 4.0, 0.05322595685720444], [' 아니요,', np.float64(8.74), np.float64(9.16), 0.05322595685720444], [' 아픈', np.float64(9.28), np.float64(9.46), 0.05322595685720444], [' 적', np.float64(9.46), np.float64(9.66), 0.05322595685720444], [' 없었어요', np.float64(9.66), np.float64(10.6), 0.05322595685720444], [' 최근에', np.float64(15.0), np.float64(16.36), 0.05322595685720444], [' 다친', np.float64(16.36), np.float64(16.7), 0.05322595685720444], [' 적이', np.float64(16.7), np.float64(16.92), 0.05322595685720444], [' 있었네?', np.float64(16.92), 18.0, 0.05322595685720444], [' 다친', np.float64(21.1), np.float64(22.46), 0.05322595685720444], [' 적', np.float64(22.46), np.float64(22.72), 0.05322595685720444], [' 없어요', np.float64(22.72), np.float64(24.28), 0.05322595685720444], [' 뭐', np.float64

## 실시간 입력 - 로컬 환경에서만 가능함.

In [5]:
import sounddevice as sd
print(sd.query_devices())


OSError: PortAudio library not found

In [4]:
import sounddevice as sd
import numpy as np
import queue
import threading
import time
from faster_whisper import WhisperModel

# Whisper 모델 불러오기
model = WhisperModel("tiny", device="cpu", compute_type="int8")

# 오디오 큐 생성
audio_queue = queue.Queue()

# 콜백 : 마이크 입력이 발생하면 큐에 넣는다.
def audio_callback(indata, frames, time, stauts):
    audio_queue.put(indata.copy())

# 음성 데이터를 적절한 길이로 누적 후 STT 수행
def audio_listener():
    buffer = []
    buffer_duration = 3  # 몇 초 간의 음성을 모아서 처리할지 (예: 3초마다 STT)
    sample_rate = 16000
    chunk_duration = 0.5  # 단위 chunk 크기 (0.5초)
    chunks_per_buffer = int(buffer_duration / chunk_duration)
    
    while True:
        chunk = audio_queue.get()
        buffer.append(chunk)
        if len(buffer) >= chunks_per_buffer:
            full_audio = np.concatenate(buffer, axis=0)
            buffer = []

            print("🎧 STT 시작...")
            segments, _ = model.transcribe(full_audio, language="ko", beam_size=5, word_timestamps=True)
            result_text = ""
            for segment in segments:
                if segment.no_speech_prob > 0.6:
                    continue
                result_text += segment.text.strip() + " "
            print("📝 인식 결과:", result_text.strip())

            # 👉 여기서 웨이크워드 감지 가능:
            if any(keyword in result_text for keyword in ["쏙닥", "지니야", "하이토끼"]):
                print("✅ 웨이크워드 감지! 대화 시작")


# 마이크 스트림 시작
def start_microphone():
    with sd.InputStream(samplerate=16000, channels=1, dtype="float32", callback=audio_callback):
        audio_listener()  # 내부에서 계속 돌면서 음성 처리

OSError: PortAudio library not found

In [ ]:
# 스레드 실행
thread = threading.Thread(target=start_microphone)
thread.start()